# Ejercicios

Los datos adjuntos a esta lección forman parte de la base de datos [NeurIPS 2020] Data Science for COVID-19 (DS4C) disponible en Kaggle. Estos datos hacen referencia a los casos de contagio de covid-19 en Corea del Sur.



El archivo csv Case contiene los casos reportados y el archivo csv PatientInfo contiene la información de los pacientes.



1. A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.

2. Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.

a. ¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.
b. A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.
c. Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

Recursos de esta clase
data.zip

In [ ]:
# Importar librerías
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession

# Crea una sesión de Spark
spark = SparkSession.builder.appName("EjercicioDataFrames").getOrCreate()

# 1. Determinar las tres ciudades con más casos confirmados
cases_df = spark.read.csv("Case.csv", header=True, inferSchema=True)
top_cities = cases_df.groupBy("province", "city").sum("confirmed").orderBy("sum(confirmed)", ascending=False).limit(3)

# Muestra el resultado
top_cities.show()

# 2. Crear un DataFrame a partir de PatientInfo.csv
patient_info_df = spark.read.csv("PatientInfo.csv", header=True, inferSchema=True)

# a. Filtrar pacientes con información de contagio
infected_patients = patient_info_df.filter(patient_info_df.infected_by.isNotNull())

# Muestra el resultado
infected_patients.show()

# b. Filtrar pacientes femeninos y seleccionar columnas
female_infected = infected_patients.filter(infected_patients.sex == "female").select(
    "patient_id", "sex", "age", "country", "province", "city", "infection_case", "infected_by", "contact_number", "symptom_onset_date", "confirmed_date", "state"
)

# Muestra el resultado
female_infected.show()

# c. Establecer número de particiones y escribir en archivo Parquet
female_infected.coalesce(2).write.mode("overwrite").partitionBy("province").parquet("data_output/female_infected.parquet")

# Detén la sesión de Spark
spark.stop()